<a href="https://colab.research.google.com/github/DellLoiD/question-answer-task/blob/main/4_%D0%94%D0%BE%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8_%D0%B4%D0%BB%D1%8F_QA_%D0%B7%D0%B0%D0%B4%D0%B0%D1%87%D0%B8_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
# Загрузка набора данных WebQuestions
dataset = tfds.load('web_questions', split='train')

# Подготовка токенизатора
tokenizer = AutoTokenizer.from_pretrained("google/t5-v1_1-Small")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/web_questions/incomplete.DVQKAC_1.0.0/web_questions-train.tfrecord*...:   …

...

tokenizer_config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# Отображение первых 5 примеров из датасета
for example in dataset.take(5):
    print(example)

{'answers': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Joe Flacco'], dtype=object)>, 'question': <tf.Tensor: shape=(), dtype=string, numpy=b'who is the ravens quarterback 2012?'>, 'url': <tf.Tensor: shape=(), dtype=string, numpy=b'http://www.freebase.com/view/en/baltimore_ravens'>}
{'answers': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Whig Party'], dtype=object)>, 'question': <tf.Tensor: shape=(), dtype=string, numpy=b'what political party was henry clay?'>, 'url': <tf.Tensor: shape=(), dtype=string, numpy=b'http://www.freebase.com/view/en/henry_clay'>}
{'answers': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'UTC+07:00'], dtype=object)>, 'question': <tf.Tensor: shape=(), dtype=string, numpy=b'where to stay in thailand koh samui?'>, 'url': <tf.Tensor: shape=(), dtype=string, numpy=b'http://www.freebase.com/view/en/ko_samui'>}
{'answers': <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'Bahrain', b'Afghanistan', b'Bangladesh', b'Bhutan', b'Brunei',
    

In [ ]:
def preprocess_text(text):
    return tokenizer.encode_plus(
        text, add_special_tokens=True,
        max_length=128, padding="max_length",
        truncation=True, return_tensors="tf")

# Функция подготовки выборки
def prepare_dataset(dataset):
    input_ids = []
    attention_masks = []
    labels = []
    for example in dataset:
        # Токенизация вопроса
        tokenized_question = preprocess_text(example['question'].numpy().decode())
        # Добавляем данные
        input_ids.append(tokenized_question["input_ids"])
        attention_masks.append(tokenized_question["attention_mask"])
        # Токенизация ответа
        answer = preprocess_text(example['answers'][0].numpy().decode())["input_ids"]
        labels.append(answer)

    return {
        'input_ids': tf.concat(input_ids, axis=0),
        'attention_mask': tf.concat(attention_masks, axis=0),
        'labels': tf.concat(labels, axis=0)
    }

In [ ]:
# Готовим тренировочные данные
prepared_data = prepare_dataset(dataset)

# Проверка размеров данных
print(f"Input IDs shape: {prepared_data['input_ids'].shape}")
print(f"Attention Mask shape: {prepared_data['attention_mask'].shape}")
print(f"Labels shape: {prepared_data['labels'].shape}")

# Архитектура трансформера на основе pre-trained модели
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/t5-v1_1-Small")

Input IDs shape: (3778, 128)
Attention Mask shape: (3778, 128)
Labels shape: (3778, 128)


tf_model.h5:   0%|          | 0.00/308M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at google/t5-v1_1-Small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, weight_decay=1e-5)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)

In [ ]:
# Компиляция модели
model.compile(optimizer=optimizer, loss=loss_function)

In [ ]:
# Тренировка модели
history = model.fit(prepared_data, epochs=40, batch_size=16)

Epoch 1/40
237/237 [==============================] - 84s 352ms/step - loss: 0.2057
Epoch 2/40
237/237 [==============================] - 83s 351ms/step - loss: 0.2004
Epoch 3/40
237/237 [==============================] - 83s 350ms/step - loss: 0.1956
Epoch 4/40
237/237 [==============================] - 83s 350ms/step - loss: 0.1907
Epoch 5/40
237/237 [==============================] - 83s 351ms/step - loss: 0.1858
Epoch 6/40
237/237 [==============================] - 83s 350ms/step - loss: 0.1810
Epoch 7/40
237/237 [==============================] - 83s 350ms/step - loss: 0.1762
Epoch 8/40
237/237 [==============================] - 83s 351ms/step - loss: 0.1722
Epoch 9/40
237/237 [==============================] - 83s 352ms/step - loss: 0.1691
Epoch 10/40
237/237 [==============================] - 83s 351ms/step - loss: 0.1647
Epoch 11/40
237/237 [==============================] - 83s 350ms/step - loss: 0.1609
Epoch 12/40
237/237 [==============================] - 83s 351ms/step - lo

In [ ]:
# Тестирование модели
test_question = "THE LARGEST WAR IN THE WORLD??"
# Шаг 1: Токенизация
inputs = tokenizer(test_question, return_tensors="tf").input_ids
print(f'Inputs (Token IDs): {inputs.numpy()}')

# Шаг 2: Генерация ответа с изменением параметров
outputs = model.generate(inputs, num_beams=5, early_stopping=True)
print(f'Outputs (Generated Token IDs): {outputs.numpy()}')

# Шаг 3: Декодирование ответа
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f'Decoded Answer: {answer}')

# Шаг 4: Финальный вывод
print(f'Question: {test_question}\nAnswer: {answer}')

Inputs (Token IDs): [[ 1853  5292 12912  6038   549  4280  3388  1853     3 30944  8546     1]]
Outputs (Generated Token IDs): [[   0 1150 1602   27    1    0]]
Decoded Answer: World War I
Question: THE LARGEST WAR IN THE WORLD??
Answer: World War I
